# Nonlinear transportation model
[![nltrans_lecture.ipynb](https://img.shields.io/badge/github-%23121011.svg?logo=github)](https://github.com/ampl/amplcolab/blob/master/ampl-lecture/nltrans_lecture.ipynb) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ampl/amplcolab/blob/master/ampl-lecture/nltrans_lecture.ipynb) [![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/ampl/amplcolab/blob/master/ampl-lecture/nltrans_lecture.ipynb) [![Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://console.paperspace.com/github/ampl/amplcolab/blob/master/ampl-lecture/nltrans_lecture.ipynb) [![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/ampl/amplcolab/blob/master/ampl-lecture/nltrans_lecture.ipynb)

Description: book example autogenerated using nltransd.mod, nltrans.dat, and nltrans.run

Tags: ampl-only, ampl-book, nonlinear

Notebook author: Marcos Dominguez Velad <marcos@ampl.com>

Model author: N/A


In [1]:
# Install dependencies
!pip install -q amplpy

In [2]:
# Google Colab & Kaggle interagration
MODULES=['ampl', 'coin']
from amplpy import tools
ampl = tools.ampl_notebook(modules=MODULES, globals_=globals()) # instantiate AMPL object and register magics

## Nonlinear transportation model

This is a variation of the linear transportation model presented on the Chapter 3 of the AMPL book, containing a nonlinear objective. There are a set of *origin* nodes, and a set of *destination* nodes (net model).

* Sets:
    * `ORIG`: origin nodes
    * `DEST`: final nodes
* Parameters:
    * `supply {ORIG}`: available units at origins
    * `demand {DEST}`: required units at destinations
    * `limit {ORIG,DEST}`: maximum capacity on routes between two nodes
    * `rate {ORIG,DEST}`: base shipment costs per unit
* Variables:
    * `Trans {ORIG,DEST}`: units to be shipped
* Objective: minimize total cost (**nonlinear**)

$$\sum \limits_{\substack{i \in ORIG \\ j \in DEST}} rate[i,j] \cdot \frac{Trans[i,j]}{1 - \frac{Trans[i,j]}{limit[i,j]}}$$

The bigger the `Trans[i,j]` value, the closer to `limit[i,j]` (upper bound) so denominator tends to $1-1=0$ implying high costs.

* Constraints:
    * `Supply {ORIG}`: node ships units equal to supply capacity:
    
    $$\sum \limits_{j \in DEST} Trans[i,j] = supply[i]$$
    
    * `Demand {DEST}`: node gets units equal to demand:
    
    $$\sum \limits_{i \in ORIG} Trans[i,j] = demand[j]$$

In [3]:
%%writefile nltrans.mod
set ORIG;   # origins
set DEST;   # destinations

param supply {ORIG} >= 0;   # amounts available at origins
param demand {DEST} >= 0;   # amounts required at destinations

   check: sum {i in ORIG} supply[i] = sum {j in DEST} demand[j];

param rate {ORIG,DEST} >= 0;   # base shipment costs per unit
param limit {ORIG,DEST} > 0;   # limit on units shipped

var Trans {i in ORIG, j in DEST} >= 0;    # actual units to be shipped

minimize Total_Cost:
   sum {i in ORIG, j in DEST}
      rate[i,j] * Trans[i,j] / (1 - Trans[i,j]/limit[i,j]);

subject to Supply {i in ORIG}:  
   sum {j in DEST} Trans[i,j] = supply[i];

subject to Demand {j in DEST}:  
   sum {i in ORIG} Trans[i,j] = demand[j];
   


Overwriting nltrans.mod


In [4]:
%%writefile nltrans.dat
data;

param: ORIG:  supply :=
        GARY   1400    CLEV   2600    PITT   2900 ;

param: DEST:  demand :=
        FRA     900    DET    1200    LAN     600 
        WIN     400    STL    1700    FRE    1100 
        LAF    1000 ;

param rate :  FRA  DET  LAN  WIN  STL  FRE  LAF :=
        GARY   39   14   11   14   16   82    8
        CLEV   27    9   12    9   26   95   17
        PITT   24   14   17   13   28   99   20 ;

param limit :  FRA  DET  LAN  WIN  STL  FRE  LAF :=
        GARY   500 1000 1000 1000  800  500 1000
        CLEV   500  800  800  800  500  500 1000
        PITT   800  600  600  600  500  500  900 ;


Overwriting nltrans.dat


In [5]:
%%ampl_eval
model nltrans.mod;
data nltrans.dat;
option solver ipopt;
option ipopt_options 'outlev=0';
solve;
display Total_Cost, Trans;

Ipopt 3.12.13: outlev=0


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

 
Ipopt 3.12.13: Maximum Number of Iterations Exceeded.

suffix ipopt_zU_out OUT;
suffix ipopt_zL_out OUT;
Total_Cost = -22681400000

Trans [*,*] (tr)
:      CLEV       GARY      PITT      :=
DET   518.41    279.922    401.668
FRA   326.573   153.849    419.579
FRE   377.651   371.911    350.438
LAF   409.194   173.889    416.917
LAN   284.738    34.4686   280.794
STL   500       372.77     827.229
WIN   183.433    13.1917   203.375
;



The previous problem is unbounded, however, this is not what we expect for this problem. This is due to the highly nonlinear objective function. 

Because the behavior of a nonlinear optimization algorithm can be sensitive to the choice of starting guess, we might hope that the solver will have greater success from a different start. To ensure that the comparison is meaningful, we first set

```
option send_statuses 0;
```

So that status information about variables that was returned by the previous solve will not be sent back to help determine a starting point for the next solve. We can use the `let` command to suggest a new initial value for each variable. For example, say that `Trans[i,j]` is half of `limit[i,j]`.

In [6]:
%%ampl_eval
option send_statuses 0;
let {i in ORIG, j in DEST} Trans[i,j] := limit[i,j]/2;
solve;
display Total_Cost;
display {i in ORIG, j in DEST} Trans[i,j]/limit[i,j];

Ipopt 3.12.13: outlev=0


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

 
Ipopt 3.12.13: Optimal Solution Found
Total_Cost = 1212120

Trans[i,j]/limit[i,j] [*,*] (tr)
:       CLEV       GARY       PITT      :=
DET   0.732965   0.187385   0.710404
FRA   0.589986   0.162441   0.654733
FRE   0.731      0.739444   0.729557
LAF   0.490537   0          0.56607
LAN   0.367685   0          0.509753
STL   0.939382   0.95209    0.937274
WIN   0.123449   0          0.502068
;



Now the solver gives a right solution. However, we could have had trouble in case any `Trans` variable get closer to the `limit`, what we could check by calling
```
display {i in ORIG, j in DEST} Trans[i,j]/limit[i,j];
```
We could implement a more robust formulation by giving more explicit bounds to the variables to avoid numerical issues. With the following bounds, `Trans` will be smaller that `limit`.
```
var Trans {i in ORIG, j in DEST} >= 0, <= .9999 * limit[i,j];
```

### Multiple local optima

Let's change to a concave objective function by adding a power of 0.8:

$$\sum \limits_{\substack{i \in ORIG \\ j \in DEST}} rate[i,j] \cdot \frac{Trans[i,j]^{0.8}}{1 - \frac{Trans[i,j]}{limit[i,j]}}$$


In [7]:
%%writefile nltransc.mod
set ORIG;   # origins
set DEST;   # destinations

param supply {ORIG} >= 0;   # amounts available at origins
param demand {DEST} >= 0;   # amounts required at destinations

   check: sum {i in ORIG} supply[i] = sum {j in DEST} demand[j];

param rate {ORIG,DEST} >= 0;   # base shipment costs per unit
param limit {ORIG,DEST} > 0;   # limit on units shipped

var Trans {i in ORIG, j in DEST} >= 0, <= .9999 * limit[i,j];
    # actual units to be shipped

minimize Total_Cost:
   sum {i in ORIG, j in DEST}
      rate[i,j] * Trans[i,j]^0.8 / (1 - Trans[i,j]/limit[i,j]);

subject to Supply {i in ORIG}:  
   sum {j in DEST} Trans[i,j] = supply[i];

subject to Demand {j in DEST}:  
   sum {i in ORIG} Trans[i,j] = demand[j];


Overwriting nltransc.mod


In [8]:
%%ampl_eval
reset;
model nltransc.mod;
data nltrans.dat;
option solver ipopt;
option ipopt_options 'outlev=0';
solve;
display Total_Cost, Trans;

Ipopt 3.12.13: outlev=0


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

Error evaluating "var =" definition -1: can't evaluate pow'(0,0.8).
exit value 1
<BREAK>


We got the error

*Error evaluating "var =" definition -1: can't evaluate pow'(0,0.8).*

This is due to the fact that the derivative of the exponential function at zero is infinity. We could just give an starting point away from zero, and tighten the lower bound too:
```
var Trans {i in ORIG, j in DEST} >= 1e-10, <= .9999 * limit[i,j], := limit[i,j]/2;
```

In [9]:
%%writefile nltransd.mod
set ORIG;   # origins
set DEST;   # destinations

param supply {ORIG} >= 0;   # amounts available at origins
param demand {DEST} >= 0;   # amounts required at destinations

   check: sum {i in ORIG} supply[i] = sum {j in DEST} demand[j];

param rate {ORIG,DEST} >= 0;   # base shipment costs per unit
param limit {ORIG,DEST} > 0;   # limit on units shipped

var Trans {i in ORIG, j in DEST} >= 1e-10, <= .9999 * limit[i,j], := limit[i,j]/2;
    # actual units to be shipped

minimize Total_Cost:
   sum {i in ORIG, j in DEST}
      rate[i,j] * Trans[i,j]^0.8 / (1 - Trans[i,j]/limit[i,j]);

subject to Supply {i in ORIG}:  
   sum {j in DEST} Trans[i,j] = supply[i];

subject to Demand {j in DEST}:  
   sum {i in ORIG} Trans[i,j] = demand[j];


Overwriting nltransd.mod


In [10]:
%%ampl_eval
reset;
model nltransd.mod;
data nltrans.dat;
option solver ipopt;
option ipopt_options 'outlev=0';
solve;
display Total_Cost, Trans;

Ipopt 3.12.13: outlev=0


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

 
Ipopt 3.12.13: Restoration Phase Failed.

suffix ipopt_zU_out OUT;
suffix ipopt_zL_out OUT;
Total_Cost = 354279

Trans [*,*] (tr)
:      CLEV         GARY         PITT      :=
DET   586.429   191.805         421.765
FRA   292.089    75.1764        532.735
FRE   365.333   370.173         364.494
LAF   488.914     0.0937081     510.992
LAN   298.74      0.000276411   301.26
STL   469.177   762.751         468.072
WIN    99.318     9.99922e-05   300.682
;



By adding a new parameter `alpha` that ponders the lower bound for `Trans`, we may solve the problem from different initial guesses, providing potential different solutions.

In [11]:
%%writefile nltranse.mod
set ORIG;   # origins
set DEST;   # destinations

param supply {ORIG} >= 0;   # amounts available at origins
param demand {DEST} >= 0;   # amounts required at destinations

   check: sum {i in ORIG} supply[i] = sum {j in DEST} demand[j];

param rate {ORIG,DEST} >= 0;   # base shipment costs per unit
param limit {ORIG,DEST} > 0;   # limit on units shipped

param alpha >= 0, <= 1;
var Trans {i in ORIG, j in DEST} >= 1e-10, <= .9999 * limit[i,j], := alpha*limit[i,j];
    # actual units to be shipped

minimize Total_Cost:
   sum {i in ORIG, j in DEST}
      rate[i,j] * Trans[i,j]^0.8 / (1 - Trans[i,j]/limit[i,j]);

subject to Supply {i in ORIG}:  
   sum {j in DEST} Trans[i,j] = supply[i];

subject to Demand {j in DEST}:  
   sum {i in ORIG} Trans[i,j] = demand[j];


Overwriting nltranse.mod


In [12]:
%%ampl_eval
reset;
set TOTAL_VALUES := 1..10;
model nltranse.mod;
data nltrans.dat;
option solver ipopt;
option ipopt_options 'outlev=0';
for {it in TOTAL_VALUES}{
    let alpha := 0.1*it;
    solve;
    printf "## Iteration %d\n", it;
    display alpha, Total_Cost;
}


Ipopt 3.12.13: outlev=0


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

 
Ipopt 3.12.13: Search Direction becomes Too Small.

suffix ipopt_zU_out OUT;
suffix ipopt_zL_out OUT;
## Iteration 1
alpha = 0.1
Total_Cost = 354281

Ipopt 3.12.13: outlev=0


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

 
Ipopt 3.12.13: Searc